# Simple Balance Calculations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Linear Bed Profile

In [ ]:
b0 = 3900.
s = .1
beta = .007
alpha = 3.
nu = 10. 
width = 10.
## Run-time-determined variables
dt = .01
steps = 50000
times = np.linspace(0., dt * steps, steps)
lengths = np.full(steps, 1.)
equi = np.full(steps, 2900.)
equi_steps = [2900.,4000., 2900, 2600.]
equi_i = 1
step_moment = (0., 0., 0)
e_th = np.array([[0., 0.]])

def bed_height_lin(x):
    return b0 - s * x

def bed_mean_lin(ti):
    return b0 - s * lengths[ti] / 2.

def slope_mean_lin(ti):
    return s

def ds_dl_lin(ti):
    return 0

def ice_mean_h(ti):
    return alpha / (1 + nu * slope_mean_lin(ti)) * np.sqrt(lengths[ti])

def balance_r(h):
    return beta * (h - equi[ti])

def mass_balance_lin(ti):
    return beta * (bed_mean_lin(ti) + ice_mean_h(ti) - equi[ti]) * lengths[ti] * width

def dl_dt_lin(ti, forcing=0.):
    right_t = 3. * alpha / (2. * (1 + nu * slope_mean_lin(ti))) * np.sqrt(lengths[ti])
    left_t = alpha * nu / pow(1 + nu * slope_mean_lin(ti), 2) * pow(lengths[ti], 3. / 2.) * ds_dl_lin(ti) # Slope is constant  
    # left_t =  0.
    if (np.isclose(right_t - left_t, 0.)):
        return 0.
    return pow(right_t - left_t, -1.) * (mass_balance_lin(ti) + forcing)

for i in range(len(lengths) - 1):
    lengths[i + 1] = lengths[i] + dt * dl_dt_lin(i)
    if (abs(lengths[i + 1] - lengths[i]) < .1):
        # equi[i + 1] = equi[i] * 1.16
        equi[i + 1] = equi_steps[equi_i]
        
        # Calculate e-fold time
        cur_moment = (times[i], lengths[i], i)
        _e_height = (2. / 3 ) * (cur_moment[1] - step_moment[1]) + step_moment[1]
        _e_time = times[np.argmin(np.abs(lengths[step_moment[2]:i] - _e_height)) + step_moment[2]]
        e_th = np.append(e_th, [[_e_time, _e_height]], axis=0)
        step_moment = cur_moment
        
        # Step function
        equi_i = (equi_i + 1) if (equi_i + 1 < len(equi_steps)) else 0
    else:
        equi[i + 1] = equi[i]
e_th = e_th[1:]
    
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(times, lengths)
ax1.set_ylabel("Glacier Length (m)")
ax1.set_title("Glacier Model with Constant Slope")
ax2.plot(times, equi)
ax2.plot([times[0], times[-1]], [b0, b0], 'k--')
ax2.set_xlabel("Time (years)")
ax2.set_ylabel("Equilibrium Line Height (m)")
plt.show()

print e_th

## Concave Bed Profile

In [ ]:
b0 = 3900.
ba = 0.
xl = 7000.
# s = .1
beta = .007
alpha = 3.
nu = 10. 
width = 1.
## Run-time-determined variables
dt = .1
steps = 15000
times = np.linspace(0., dt * steps, steps)
lengths = np.full(steps, 1.)
equi = np.full(steps, 2900.)
equi_steps = [2900., 2300., 2000., 1000., 1500., 1000.]
equi_i = 1
step_moment = (0., 0., 0)
e_th = np.array([[0., 0.]])

def bed_height_con(x):
    return ba + b0 * np.exp(-x / xl)

def bed_mean_con(ti):
    return ba + (xl * b0) / lengths[ti] * (1. - np.exp(-lengths[ti] / xl))

def slope_mean_con(ti):
    return b0 * (1. - np.exp(-lengths[ti] / xl)) / lengths[ti]

def ds_dl_con(ti):
    return -b0 * (1. - np.exp(-lengths[ti]/xl)) / pow(lengths[ti], 2) +\
           b0 * np.exp(-lengths[ti] / xl) / xl / lengths[ti]

def ice_mean_h(ti):
    return alpha / (1 + nu * slope_mean_con(ti)) * np.sqrt(lengths[ti])

def balance_r(h):
    return beta * (h - equi[ti])

def mass_balance_con(ti):
    return beta * (bed_mean_con(ti) + ice_mean_h(ti) - equi[ti]) * lengths[ti] * width

def dl_dt_con(ti, forcing=0.):
    right_t = 3. * alpha / (2. * (1 + nu * slope_mean_con(ti))) * np.sqrt(lengths[ti])
    left_t = alpha * nu / pow(1 + nu * slope_mean_con(ti), 2) * pow(lengths[ti], 3. / 2.) * ds_dl_con(ti)
    if (np.isclose(right_t - left_t, 0.)):
        return 0.
    return pow(right_t - left_t, -1.) * (mass_balance_con(ti) + forcing)

for i in range(len(lengths) - 1):
    lengths[i + 1] = lengths[i] + dt * dl_dt_con(i)
    if (abs(lengths[i + 1] - lengths[i]) < .1):
        # equi[i + 1] = equi[i] / 2.
        equi[i + 1] = equi_steps[equi_i]
        # Calculate e-fold time
        cur_moment = (times[i], lengths[i], i)
        _e_height = (2. / 3 ) * (cur_moment[1] - step_moment[1]) + step_moment[1]
        _e_time = times[np.argmin(np.abs(lengths[step_moment[2]:i] - _e_height)) + step_moment[2]]
        e_th = np.append(e_th, [[_e_time, _e_height]], axis=0)
        step_moment = cur_moment
        # Step function
        equi_i = (equi_i + 1) if (equi_i + 1 < len(equi_steps)) else 0
    else:
        equi[i + 1] = equi[i]
e_th = e_th[1:]
    
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(times, lengths)
ax1.set_ylabel("Glacier Length (m)")
ax1.set_title("Glacier Model with Concave Bed")
ax2.plot(times, equi)
ax2.plot([times[0], times[-1]], [b0, b0], 'k--')
ax2.set_xlabel("Time (years)")
ax2.set_ylabel("Equilibrium Line Height (m)")
plt.show()

#fig = plt.figure()
#plt.plot(np.arange(7000), bed_height(np.arange(7000)), label="Concave Bed Shape")
#plt.xlabel("Distance from top (m)")
#plt.ylabel("Height (m)")
#plt.title("Concave Bed Shape")
#plt.show()
print e_th
print min(lengths)

## Calving Flux

In [ ]:
b0 = 3900.
ba = -100.
xl = 7000.
# s = .1
beta = .007
alpha = 3.
nu = 10. 
width = 1.

# Calving
calv_p = 1.
kfrac = .3
water_d, ice_d = (1000., 917.)
## Run-time-determined variables
dt = .1
steps = 15000
times = np.linspace(0., dt * steps, steps)
lengths = np.full(steps, 1.)
equi = np.full(steps, 2900.)
equi_steps = [2900., 2300., 2000., 950., 1500., 1000.]
equi_i = 1
# e-folding
step_moment = (0., 0., 0)
e_th = np.array([[0., 0.]])

# print -xl * np.log(-ba/b0)
def water_depth_con(ti):
    return -bed_height_con(lengths[ti]) if (bed_height_con(lengths[ti]) < 0.0) else 0.0

def water_depth_lin(ti):
    return -bed_height_lin(lengths[ti]) if (bed_height_lin(lengths[ti]) < 0.0) else 0.0

def calving_depth(ti, mode='con'):
    if mode is 'con':
        w_depth = water_depth_con(ti)
    else:
        w_depth = water_depth_lin(ti)
    return -calv_p * w_depth * width * np.max([kfrac * ice_mean_h(ti), water_d / ice_d * w_depth])

for i in range(len(lengths) - 1):
    lengths[i + 1] = lengths[i] + dt * dl_dt_con(i, calving_depth(i))
    if (abs(lengths[i + 1] - lengths[i]) < .01 and (step_moment[1] != times[i])):
        # equi[i + 1] = equi[i] / 2.
        equi[i + 1] = equi_steps[equi_i]
        # Calculate e-fold time
        cur_moment = (times[i], lengths[i], i)
        _e_height = (2. / 3 ) * (cur_moment[1] - step_moment[1]) + step_moment[1]
        _e_time = times[np.argmin(np.abs(lengths[step_moment[2]:i] - _e_height)) + step_moment[2]]
        e_th = np.append(e_th, [[_e_time, _e_height]], axis=0)
        step_moment = cur_moment
        # Step function
        equi_i = (equi_i + 1) if (equi_i + 1 < len(equi_steps)) else 0
    else:
        equi[i + 1] = equi[i]
e_th = e_th[1:]

f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(times, lengths)
ax1.set_ylabel("Glacier Length (m)")
ax1.set_title("Glacier Model with Concave Bed")
ax2.plot(times, equi)
ax2.plot([times[0], times[-1]], [b0, b0], 'k--')
ax2.set_ylabel("Equilibrium Line Height (m)")
ax2.set_xlabel("Time (years)")
plt.show()

# Check calving depth
lengths = np.linspace(1., 50000., 15000)
_waterd = np.empty(steps)
_calving = np.empty(steps)
_comb = np.empty((2, steps))
for i in range(len(lengths)):
    _waterd[i] = water_depth_con(i)
    _calving[i] = calving_depth(i)
    
    #w_depth = _waterd[i]
    _comb[0,i] = -calv_p * _waterd[i] ** 2 * width *  water_d / ice_d
    _comb[1,i] = -calv_p * _waterd[i] * width *  kfrac * ice_mean_h(i)
    
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(times, _waterd)
ax1.set_ylabel("Water Depth (m)")
ax2.plot(times, _calving)
ax2.set_ylabel("Calving Forcing (m$^3$/yr)")
ax2.set_xlabel("Time (years)")
plt.show()


f = plt.figure()
plt.plot(lengths, _comb[0,:], label='$\\frac{\\rho_w}{\\rho_i}d$')
plt.plot(lengths, _comb[1,:], label='$\\kappa H_m$')
plt.xlabel("Glacier Length (m)")
plt.ylabel("Calving Forcing (m$^3$/yr)")
plt.legend()
plt.show()

#_ice_mean_h = np.array([ice_mean_h(i) for i in range(len(lengths))])
#plt.plot(times, _ice_mean_h)
#plt.show()

#fig = plt.figure()
#plt.plot(np.arange(7000), bed_height(np.arange(7000)), label="Concave Bed Shape")
#plt.xlabel("Distance from top (m)")
#plt.ylabel("Height (m)")
#plt.title("Concave Bed Shape")
#plt.show()
print e_th

## ELS Data from external files

In [ ]:
def read_els_data(filename):
    import os.path
    assert(os.path.exists(filename))
    return np.genfromtxt(filename, delimiter='\t')

### Concave Bed

In [ ]:
b0 = 870.
ba = 0.
xl = 7000.
s = 1.7e-2
beta = .007
alpha = 3.
nu = 10. 
width = 1.

## Run-time-determined variables
dt = 1.
steps = 4600
times = np.linspace(0., dt * steps, steps)
lengths = np.full(steps, 1.)

equi_per= read_els_data(".\\ELS_data_spitsbergen.txt")
equi = np.full(steps, equi_per[0,1])
equi_steps = equi_per.T[1][0:2]

# Custom Geometry
elevation = np.array([1100, 1000, 900, 800, 700, 600, 500, 400, 300, 200, 100, 0])
position  = np.array([0.0, 4700.0, 9400.0, 14300.0, 19000.0, 23600.0, 28100.0, 32500.0, 36000.0, 38700.0, 40900.0, 44900.0])

#coordinates = [[17.51806823248436,79.35771072001703], [17.34105444999151,79.33954934516657],
#               [17.28735106260403,79.28482535062538], [17.28132213793325,79.22592416747455],
#               [17.31551766413089,79.1776906844552 ], [17.31885134701311,79.14110861124895],
#               [17.22565063381898,79.11049960734091], [17.1947585771082, 79.06589866869315 ],
#               [17.21567039201253,79.02996481586489], [17.26296874145393,78.99655772735513]]
#coordinates = np.transpose(coordinates)
#coordinates = np.array([coordinates[1], coordinates[0]])
#np.savetxt("veteranen_coords.csv", coordinates, delimiter=',')
#heights = [geocoder.elevation(coord).meters for coord in coordinates]
# e-folding
print max(equi_per.T[1])

def calc_e_folding_t(i, stp_moment, e_list):
    # Calculate e-fold time
    cur_moment = (times[i], lengths[i], i)
    _e_height = (2. / 3 ) * (cur_moment[1] - stp_moment[1]) + stp_moment[1]
    _e_time = times[np.argmin(np.abs(lengths[stp_moment[2]:i] - _e_height)) + stp_moment[2]]
    e_list = np.append(e_list, [[_e_time, _e_height]], axis=0)
    return (cur_moment, e_list)

def bed_height_cust(ti, dists, els):
    return np.interp(lengths[ti], dists, els)
        
def bed_mean_cust(ti, dists, els):
    if np.isclose(lengths[ti], .0):
        return 0.
    ti = max(ti, 1) # BUG FIX otherwise heights is empty array
    #if ti is 0:
    #    return 0.
    #interv = lengths[:ti]
    #heights = np.array([np.interp(lengths[tx], dists, els) for tx in range(ti)])
    heights = np.array([np.interp(l, dists, els) for l in lengths])
    return np.sum(heights[:ti] * (lengths[1] - lengths[0])) / lengths[ti]

def slope_mean_cust(ti, dists, els):
    if np.isclose(lengths[ti], .0):
        return 0.
    ti = max(ti, 1) # BUG FIX otherwise dh_dx is empty array
    #heights = np.array([np.interp(lengths[tx], dists, els) for tx in range(ti)])
    #if ti is 1:
    #    dh_dx = -np.diff(heights) / (lengths[1] - lengths[0])
    #    print ti
    #    print heights
    #else:
    #    dh_dx = -np.gradient(heights, lengths[2] - lengths[0])
    heights = np.array([np.interp(l, dists, els) for l in lengths])
    dh_dx = -np.gradient(heights, lengths[1] - lengths[0])[:ti]
    return np.sum(dh_dx * (lengths[1] - lengths[0])) / lengths[ti]

def ds_dl_cust(ti, dists, els):
    if np.isclose(lengths[ti], .0):
        return 0.
    elif ti is 0:
        # Use right differences at beginning of length array
        return (slope_mean_cust(ti + 1, dists, els) - slope_mean_cust (ti, dists, els)) /\
               (lengths[ti + 1] - lengths[ti])
    elif ti is (len(lengths) - 1):
        # Use left differences at end of length array
        return (slope_mean_cust(ti, dists, els) - slope_mean_cust (ti - 1, dists, els)) /\
               (lengths[ti] - lengths[ti - 1])
    # Otherwise central differences
    return (slope_mean_cust(ti + 1, dists, els) - slope_mean_cust (ti - 1, dists, els)) /\
           (lengths[ti + 1] - lengths[ti - 1])

def mass_balance_cust(ti):
    return beta * (bed_mean_cust(ti) + ice_mean_h(ti) - equi[ti]) * lengths[ti] * width

def run_simulation(bed_profile='con', equi_shape=None, use_calving=False):
    equi_i = 1
    step_moment = (0., 0., 0)
    e_th = np.array([[0., 0.]])
    
    for i in range(len(lengths) - 1):
        if type(bed_profile) is str:
            forcing = calving_depth(i, bed_profile) if use_calving else 0.
            dl_dt = dl_dt_con(i, forcing) if bed_profile is 'con' else dl_dt_lin(i, forcing)
        else:
            forcing = 0.
            dl_dt = dl_dt_cust(i, forcing)
        
        lengths[i + 1] = lengths[i] + dt * dl_dt
        if (abs(lengths[i + 1] - lengths[i]) < .1) and (equi_shape is None):
            equi[i + 1] = equi_steps[equi_i] 
            # Calculate e-fold time
            step_moment, e_th = calc_e_folding_t(i, step_moment, e_th)
            # Step function
            equi_i = (equi_i + 1) if (equi_i + 1 < len(equi_steps)) else 0
        elif (equi_shape is not None):
            t_offset = i * dt + equi_shape[0,0]
            t_index = 0
            while equi_shape[t_index, 0] < t_offset and (t_index < (equi_shape.shape[0] - 1)):
                t_index = t_index + 1
            equi[i + 1] = equi_shape[t_index][1]
        else:
            equi[i + 1] = equi[i]
    return e_th[1:]

#run_simulation('con', equi_per)
# Calculate Stable height for first value
start_h = run_simulation('con').T[1,-1]
lengths = np.full(steps, start_h)
run_simulation('con', equi_per)
    
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(times, lengths)
ax1.set_ylabel("Glacier Length (m)")
ax1.set_title("Glacier Model with Concave Bed")
ax2.plot(times, equi)
ax2.plot([times[0], times[-1]], [b0, b0], 'k--')
ax2.set_xlabel("Time (years)")
ax2.set_ylabel("Equilibrium Line Height (m)")
plt.show()

# Exploration of Different Bed Profiles

In [ ]:
b0 = max(elevation)
s = (max(elevation) - min(elevation)) / (max(position))
xl = 7000.

lengths = np.linspace(1., max(position), 100)
cust_bed = [bed_height_cust(ix, position, elevation) for ix in range(len(lengths))]
mcust = [bed_mean_cust(ix, position, elevation) for ix in range(len(lengths))]
scust = np.array([slope_mean_cust(ix, position, elevation) for ix in range(len(lengths))])
dsdLcust = np.array([ds_dl_cust(ix, position, elevation) for ix in range(len(lengths))])

f, ax1 = plt.subplots()
leg1, = ax1.plot(lengths * 1e-3, cust_bed, 'b', label='Bed Height')
leg2, = ax1.plot(lengths * 1e-3, mcust, 'b--', label='Mean Bed Height')
ax2 = ax1.twinx()
leg3, = ax2.plot(lengths * 1e-3, scust, 'r', label='Mean Bed Slope')
leg4, = ax2.plot(lengths * 1e-3, dsdLcust, 'r--', label='$\\mathrm{d}s/\\mathrm{d}L$')
ax1.set_xlabel("Length (km)")
ax1.set_ylabel("Height (m)")
ax2.set_ylabel("Slope and $\\mathrm{d}s/\\mathrm{d}L (m^{-1}$)")
ax1.legend(handles=[leg1, leg2, leg3, leg4], bbox_to_anchor=(1.2, 1), loc=2, borderaxespad=0.)
ax1.grid(True)
plt.title("Custom Profile")
plt.show()

f, ax1 = plt.subplots()
leg1, = ax1.plot(lengths * 1e-3, [bed_height_lin(x) for x in lengths], 'b', label='Linear Bed Height')
leg2, = ax1.plot(lengths * 1e-3, [bed_mean_lin(x) for x in range(lengths.size)], 'b--', label='Linear Mean Height')
ax2 = ax1.twinx()
leg3, = ax2.plot(lengths * 1e-3, [slope_mean_lin(x) for x in range(lengths.size)], 'r', label='Linear Mean Slope')
leg4, = ax2.plot(lengths * 1e-3, [ds_dl_lin(x) for x in range(lengths.size)], 'r--', label='Linear $\\mathrm{d}s/\\mathrm{d}L$')
ax1.set_xlabel("Length (km)")
ax1.set_ylabel("Height (m)")
ax2.set_ylabel("Slope and $\\mathrm{d}s/\\mathrm{d}L$ (m$^{-1}$)")
plt.legend(handles=[leg1, leg2, leg3, leg4], bbox_to_anchor=(1.2, 1), loc=2, borderaxespad=0.)
ax1.grid(True)
plt.title("Linear Bed Profile")
plt.show()

f, ax1 = plt.subplots()
leg1, = ax1.plot(lengths * 1e-3, [bed_height_con(x) for x in lengths], 'b', label='Concave Bed Height')
leg2, = ax1.plot(lengths * 1e-3, [bed_mean_con(x) for x in range(lengths.size)], 'b--', label='Concave Mean Height')
ax2 = ax1.twinx()
leg3, = ax2.plot(lengths * 1e-3, [slope_mean_con(x) for x in range(lengths.size)], 'r', label='Concave Mean Slope')
leg4, = ax2.plot(lengths * 1e-3, [ds_dl_con(x) for x in range(lengths.size)], 'r--', label='Concave $\\mathrm{d}s/\\mathrm{d}L$')
ax1.set_xlabel("Length (km)")
ax1.set_ylabel("Height (m)")
ax2.set_ylabel("Slope and $\\mathrm{d}s/\\mathrm{d}L$ (m$^{-1}$)")
plt.legend(handles=[leg1, leg2, leg3, leg4], bbox_to_anchor=(1.2, 1), loc=2, borderaxespad=0.)
ax1.grid(True)
plt.title("Concave Bed Profile")
plt.show()